In [1]:
import os
import numpy as np
import time
import pandas as pd
import cv2 
import sys
import time
import pandas as pd
from PIL import Image
import subprocess as sp
import multiprocessing as mp
import timeit
import PySimpleGUI as sg
from datetime import datetime
from cryptography.fernet import Fernet

In [2]:
class FaceRecognition:
    def __init__(self,name):
        self.name = name
        
    def FR(self):
        faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        cap = cv2.VideoCapture(0) #camera input
        cap.set(3,640) # set Width
        cap.set(4,480) # set Height

        while True:
            ret, img = cap.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(
                gray,     
                scaleFactor=1.3,
                minNeighbors=5,     
                minSize=(20, 20)
            )
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = img[y:y+h, x:x+w] 
        
            cv2.imshow('video',img)
    
            k = cv2.waitKey(30) & 0xff
            if k == 27: # press 'ESC' to quit
                break
        
        cap.release()
        cv2.destroyAllWindows()

In [3]:
class FaceCollection:
    
    def __init__(self,name):
        self.name = name
    
    def FC(self,path):
        cam = cv2.VideoCapture(0) # camera input
        cam.set(3, 640) # set video width
        cam.set(4, 480) # set video height
        face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        # For each person, enter one numeric face id
        face_id = input('\n Wybierz numer ID, jaki chcesz przypisac do uzytkownika: ')
        imie = input('\n Wpisz swoje imię: ')
        print("\n Wczytywanie modułu..Proszę patrz prosto w kamerę...")
        time.sleep(3)
        # Initialize individual sampling face count

        folder_dir = path +str(imie) +'\\'

        isExist = os.path.exists(folder_dir)

        if not isExist:
  
          # Create a new directory because it does not exist 
            os.makedirs(folder_dir)
            print('\n Utworzono nowy folder!')

        #print(folder_dir)
        count = 0
        while(True):
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_detector.detectMultiScale(gray, 1.3, minSize=(20,20))
            for (x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
                count += 1
        
                # Save the captured image into the datasets folder
                cv2.imwrite(folder_dir+"ID_"+ str(face_id) + "_zdjecie_" +  
                            str(count) + ".jpg", gray[(y-20):(y+h+20),(x-20):(x+w+20)])
                cv2.imshow('image', img)
            k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
            if k == 27:
                break
            elif count >= 50: 
                 break

        time.sleep(2)
        print("\n Zapisywanie danych i czyszczenie pamięci programu...")
        time.sleep(1)
        cam.release()
        cv2.destroyAllWindows()

In [4]:
class FaceTraining:
    
    def __init__(self,name):
        self.name = name
        
    def FT(self,path):
        face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
        # function to get the images and label data
        def getImagesAndLabels(path):
            imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
            faceSamples=[]
            ids = []
            for imagePath in imagePaths:
                PIL_img = Image.open(imagePath).convert('L') # grayscale
                img_numpy = np.array(PIL_img,'uint8')
                id = int(os.path.split(imagePath)[-1].split("_")[1])
                faces = detector.detectMultiScale(img_numpy)
                for (x,y,w,h) in faces:
                    faceSamples.append(img_numpy[y:y+h,x:x+w])
                    ids.append(id)
            return faceSamples,ids
        print ("\n Program trenuje dostarczone dane. To może zająć chwile..Poczekaj...")
        faces,ids = getImagesAndLabels(path)
        face_recognizer.train(faces, np.array(ids))
        # Save the model into trainer/trainer.yml
        face_recognizer.write('trainer.yml') 
        # Print the numer of faces trained and end program
        print("\n Obiekty wytrenowano. Zamykanie programu")


In [5]:
class RealFaceTimeRecognition():
    
    def __init__(self,name):
        self.name = name
    
    def RFTR(self):
        
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read('trainer.yml')
        cascadePath = "Cascades/haarcascade_frontalface_default.xml"
        faceCascade = cv2.CascadeClassifier(cascadePath);
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        #iniciate id counter
        id = 0
        ID_to_verify = []
        key_value = 0
        # names related to ids: 
        names = ['Niezidentyfikowany!!','Jakub', 'Robert Kubica', 'X', 'Y'] 
        
        # Initialize and start realtime video capture
        cam = cv2.VideoCapture(2) # camera input
        cam.set(3, 640) # set video widht640
        cam.set(4, 480) # set video height480
        
        timeout = 10   # [seconds]
        timeout_start = time.time()

        while True:
            ret, img =cam.read()
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

            faces = faceCascade.detectMultiScale( 
                gray,
                scaleFactor = 1.2,
                minNeighbors = 5,
                minSize = (30,30),
               )
            
            for(x,y,w,h) in faces:                
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
                id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

                if (confidence < 100):
                    id = names[id]
                    confidence = "  {0}%".format(round(confidence))

                else:
                    id = "Nieznany!!"
                    cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 2)
                    confidence = "  {0}%".format(round(100-confidence))


                cv2.putText(
                            img, 
                            str(id), 
                            (x+5,y-5), 
                            font, 
                            1, 
                            (255,255,255), 
                            2
                           )
                ID_to_verify.append(id)
                cv2.putText(
                            img, 
                            str(confidence), 
                            (x+5,y+h-5), 
                            font, 
                            1, 
                            (255,255,0), 
                            1
                           )  


            cv2.imshow('Identity verification',img)

            k = cv2.waitKey(30) & 0xff
            if k == 27 or time.time() > timeout_start + timeout : # press 'ESC' to quit
                break

        count = pd.Series(ID_to_verify).value_counts()
        result = count.index[0]
        print("\n",count.index[0],": ", count[0]) 
        #print("\n",count.index[1],": ", count[1])

        if result =="Jakub":
            key_value = bool(1)
        else:
            key_value =bool(0)

        if key_value == 1:
            print("\n Gratulacje uzyskałeś dostęp!!")
        else:
            print("\n Brak dostępu, niezidentyfikowany użytkownik!!")
            print("\n Zakończanie programu i czyszczenie pamięci..")

        cam.release()
        cv2.destroyAllWindows()
        
        return key_value

In [6]:
class SimpleEncription:
    
    def createKey(self,key_name):
        key = Fernet.generate_key()

        with open(key_name, 'wb') as mykey:
            mykey.write(key)
            
    def loadKey(self,key_name):
        with open(key_name, 'rb') as mykey:
            key = mykey.read()
        return key
        
    def fileEncryption(self, key, original_file, encrypted_file):
        
        f_key = Fernet(key)

        with open(original_file, 'rb') as file:
            original = file.read()

        encrypted = f_key.encrypt(original)

        with open (encrypted_file, 'wb') as file:
            file.write(encrypted)
    
    def fileDecryption(self, key, encrypted_file, decrypted_file):
        
        f_key = Fernet(key)

        with open(encrypted_file, 'rb') as file:
            encrypted = file.read()

        decrypted = f_key.decrypt(encrypted)

        with open(decrypted_file, 'wb') as file:
            file.write(decrypted)
               

In [13]:
#PROGRAM miniSEJFU ROZPOZNAJĄCY TWARZ KONKRETNEGO UŻYTKOWNIKA
sg.theme('Dark Brown3')

layout = [
    [sg.Text("miniSEJF ~~ Twoje dane za bezpieczną barierą!",font =('Helvetica', 15,'bold'))],
    [sg.Text("Wybrany profil:",auto_size_text = False, size=(12),font =('Helvetica', 14)),
     sg.Combo(["Jakub"],default_value="None", auto_size_text=False, size = (20),key ="-PROFIL-")],
    [sg.Button("Zatwierdź profil",auto_size_button=False, size = (15,2), key = "-SZUKAJ-"),
     sg.Button("Otwórz sejf",visible = False, auto_size_button=False, size = (20,2), key = "-SEJF-")],
    [sg.HorizontalSeparator(color = "Light Blue")],
    [sg.Multiline(default_text=' Dzień dobry!\n \n Jest godzina: '+datetime.now().strftime("%H:%M:%S"),
                  enable_events= True ,horizontal_scroll = False, font =20, size=(50,5), visible = True,key = "-ZAWARTOSC-")],
    [sg.Button("Zamknij okno", auto_size_button=False, size = (15,2),button_color= 'Brown')]
        
]

datetime.now().strftime("%H:%M:%S")
        
window = sg.Window("miniSEJF", layout,location =(660,200),size=(500,285),keep_on_top=False, disable_minimize = False)

while True:
    event, values = window.read() 
    
    if event == "Zamknij okno" or event == sg.WIN_CLOSED:
        break
    
    if event == "-SZUKAJ-"and values["-PROFIL-"]=="Jakub":
        
        window["-ZAWARTOSC-"].update('Wybrałeś profil:  '+ values["-PROFIL-"] )
        window.refresh()
        window["-SEJF-"].update(visible=True)
        
    if event ==  "-SEJF-":
        
        window["-ZAWARTOSC-"].update(' Uwaga!!..\n Wczytuję moduł rozpoznawania twarzy wybranego użytkownika..\n Po załadowaniu modułu patrz prosto w kamerę \n przez około 10 sekund..')
        window.refresh()
        
        time.sleep(5)
        faceRecognition = RealFaceTimeRecognition("new")
        rezults = faceRecognition.RFTR()
        
        if rezults== True:
            window["-ZAWARTOSC-"].update(' Gratulację, uzyskałeś dostęp do sejfu!! \n Możesz teraz edytować swój dokument :)\n \n Po zakończeniu pracy w "sejfie" zamknij program!')
            window.refresh()
            
            encryptor=SimpleEncription()
            key = encryptor.loadKey("xyz")
            encryptor.fileDecryption(key,"zaszyfrowanySEJF.txt","SEJF.txt")
            time.sleep(1)
            sp.Popen(["Notepad.exe","SEJF.txt"])
           
            
        else:
            window["-ZAWARTOSC-"].update(' PRÓBA NIEAUTORYZOWANEGO DOSTĘPU!! ')
            sg.PopupError("Nieautoryzowany użytkownik, odmowa dostępu!",title = "Błąd dostępu!",auto_close = True,location=(800,300))
            window.refresh()
            time.sleep(1)
            break  
            
file_dir = r'C:\Users\jakub\Desktop\Staff_na_studia\Semestr 7\Obiektowe\SEJF.txt'
isExist = os.path.exists(file_dir)

if isExist:
    encryptor.fileEncryption(key,"SEJF.txt","zaszyfrowanySEJF.txt")
    time.sleep(1)
    os.remove("SEJF.txt")

window.close()


 Jakub :  101

 Gratulacje uzyskałeś dostęp!!
